In [1]:
from gurobipy import *
import pandas as pd
import numpy as np
import os

# display settings
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.set_option('display.float_format', lambda x: '%.4f' % x)

## Set mode

In [9]:
mode = "train"
# mode = "testcase"

## Set working directory

In [10]:
print(os.getcwd())

if mode == "train":
    if (os.getcwd() == "/Users/wangyanpu/Desktop/台灣大學/大三下/作業研究/case/Case2/codes_ta/data"):
        pass
    else:
        os.chdir("/Users/wangyanpu/Desktop/台灣大學/大三下/作業研究/case/Case2/codes_ta/data")
else:
    if (os.getcwd() == "/Users/wangyanpu/Desktop/台灣大學/大三下/作業研究/case/Case2/testcase/testcases"):
        pass
    else:
        os.chdir("/Users/wangyanpu/Desktop/台灣大學/大三下/作業研究/case/Case2/testcase/testcases")

print(os.getcwd())
os.listdir()

/Users/wangyanpu/Desktop/台灣大學/大三下/作業研究/case/Case2/testcase/testcases
/Users/wangyanpu/Desktop/台灣大學/大三下/作業研究/case/Case2/codes_ta/data


['instance 1.csv',
 'instance 3.csv',
 'instance 2.csv',
 'instance 5.csv',
 'instance 4.csv']

## Defining Functions

In [11]:
def split_(row, machine_set):
    if row is np.nan:
        return list(machine_set)
    else:
        return list(map(int, row.split(",")))

In [12]:
def file_preprocess(file_path):
    ''' read csv and create usable dataframe'''
    
    # get machine set
    df = pd.read_csv(file_path, index_col=0)
    mfor1 = df['Stage-1 Machines'].values
    mfor2 = df['Stage-2 Machines'].values
    mfor1 = [list(map(int, x.split(','))) for x in mfor1]
    mfor2 = [list(map(int, x.split(','))) for x in mfor2 if x is not np.nan]
    mfor1 = [item for sublist in mfor1 for item in sublist]
    mfor2 = [item for sublist in mfor2 for item in sublist]
    machine_set_ = list(set(mfor1 + mfor2))
    
    # turn strings into list in dataframe
    df["Stage-1 Machines"] = df["Stage-1 Machines"].apply(split_, machine_set=machine_set_)
    df["Stage-2 Machines"] = df["Stage-2 Machines"].apply(split_, machine_set=machine_set_)
    
    from sklearn.preprocessing import MultiLabelBinarizer
    stage1 = df["Stage-1 Machines"]
    stage2 = df["Stage-2 Machines"]

    # create stage machine dummy variables from list
    mlb = MultiLabelBinarizer()
    mlb2 = MultiLabelBinarizer()
    dummyM1_df = pd.DataFrame(mlb.fit_transform(stage1), columns=mlb.classes_, index=df.index)
    dummyM1_df = dummyM1_df.add_prefix("stage1_m")
    dummyM2_df = pd.DataFrame(mlb.transform(stage2), columns=mlb.classes_, index=df.index)
    dummyM2_df = dummyM2_df.add_prefix("stage2_m")
    dummyM1_df, dummyM2_df
    
    # dummy column names
    dummyM1_col = dummyM1_df.columns
    dummyM2_col = dummyM2_df.columns

    df = df.join(dummyM1_df, on="Job ID")
    df = df.join(dummyM2_df, on="Job ID")
    df = df.drop(["Stage-1 Machines", "Stage-2 Machines"], axis=1)
#     display(df)
    
    return df, machine_set_, dummyM1_col, dummyM2_col

In [13]:
def IP(instance, machine_set, dummyM1_col, dummyM2_col, time_limit1, time_limit2):
    ''' two stage optimization'''
    
    # turn data into corresponding list
    pt_stage1 = instance["Stage-1 Processing Time"].values
    pt_stage2 = instance["Stage-2 Processing Time"].values
    due_time = instance["Due Time"].values

    job_id = list(instance.index)
    m_dummy_stage1 = instance[dummyM1_col].values
    m_dummy_stage2 = instance[dummyM2_col].values
#     print(m_dummy_stage1)
#     print(m_dummy_stage1[0])
#     print(m_dummy_stage2)
#     print(m_dummy_stage2[0])
    
    # L: big number
    L = (sum(pt_stage1) + sum(pt_stage2)) * 100
    
    ''' stage 1 '''
    p1 = Model("p1")
    p1.setParam('TimeLimit', time_limit1)
    j_num = int(len(job_id))
    m_len = len(machine_set)

    # Variables
    s_jkm = []
    c_jkm = []
    x_jkm = []
    for j in range(j_num):
        tmp_s = []
        tmp_c = []
        tmp_x = []
        for k in range(2):
            tmp_s.append([])
            tmp_c.append([])
            tmp_x.append([])
            for m in machine_set:
                tmp_s[k].append(p1.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "s_" + str(j + 1) + "," + str(k + 1) + "," + str(m)))
                tmp_c[k].append(p1.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "c_" + str(j + 1) + "," + str(k + 1) + "," + str(m)))
                tmp_x[k].append(p1.addVar(lb = 0, vtype = GRB.BINARY, name = "x_" + str(j + 1) + "," + str(k) + ","+ str(m)))  
        s_jkm.append(tmp_s)
        c_jkm.append(tmp_c)
        x_jkm.append(tmp_x)

    y_jkjkm = []
    for j1 in range(j_num):
        y_jkjkm.append([])
        for k1 in range(2):
            y_jkjkm[j1].append([])
            for j2 in range(j1 + 1, j_num):
                y_jkjkm[j1][k1].append([])
                for k2 in range(2):
                    y_jkjkm[j1][k1][j2 - j1 - 1].append([])
                    for m in machine_set:
                        y_jkjkm[j1][k1][j2 - j1 - 1][k2].append(p1.addVar(lb = 0, vtype = GRB.BINARY, 
                                                       name = "y_" + str(j1 + 1) + "," + str(k1 + 1) + "," + str(j2 + 1) + "," + str(k2 + 1)+ "," + str(m)))

    c_j = []
    for j in range(j_num):
        c_j.append(p1.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "c_" + str(j + 1)))
    t_j = []
    for j in range(j_num):
        t_j.append(p1.addVar(lb = 0, vtype = GRB.BINARY, name = "t_" + str(j + 1)))
        
    # setting the objective function 
    p1.setObjective(quicksum(t_j[j] for j in range(j_num)), GRB.MINIMIZE) 
    
    # add constraints
    # job stage 1 machine limit
    p1.addConstrs((x_jkm[j][0][m] <= m_dummy_stage1[j][m] for j in range(j_num) for m in range(m_len)))
    # job stage 2 machine limit
    p1.addConstrs((x_jkm[j][1][m] <= m_dummy_stage2[j][m] for j in range(j_num) for m in range(m_len)))

    # tardy variable
    p1.addConstrs((t_j[j] * L >= c_j[j] - due_time[j] 
                    for j in range(j_num)), "tardy count")

    # job complete time
    p1.addConstrs((c_j[j] >= quicksum(c_jkm[j][1][m] for m in range(m_len))
                    for j in range(j_num)), "job completion time")

    # each job assigned once
    p1.addConstrs((quicksum(x_jkm[j][k][m] for m in range(m_len)) == 1 for j in range(j_num) for k in range(2)), "job assignment constraint")

    p1.addConstrs((s_jkm[j][k][m] + c_jkm[j][k][m] <= x_jkm[j][k][m] * L for j in range(j_num) for k in range(2) for m in range(m_len)))

    # process time
    p1.addConstrs((c_jkm[j][0][m] >= s_jkm[j][0][m] + pt_stage1[j] - (1 - x_jkm[j][0][m]) * L for j in range(j_num) for m in range(m_len)))
    p1.addConstrs((c_jkm[j][1][m] >= s_jkm[j][1][m] + pt_stage2[j] - (1 - x_jkm[j][1][m]) * L for j in range(j_num) for m in range(m_len)))

    for m in range(m_len):
        for j1 in range(j_num):
            for k1 in range(2):
                for j2 in range(j_num - j1 - 1):
                    for k2 in range(2):
                    # 注意ijm的indexing方法不同
                        p1.addConstr(s_jkm[j1][k1][m] >= c_jkm[j1 + j2 + 1][k2][m] - y_jkjkm[j1][k1][j2][k2][m] * L)
                        p1.addConstr(s_jkm[j1 + j2 + 1][k2][m] >= c_jkm[j1][k1][m] - (1 - y_jkjkm[j1][k1][j2][k2][m]) * L)

    # subjob 2 starts only after subjob 1
    p1.addConstrs((quicksum(s_jkm[j][1][m] for m in range(m_len)) >= quicksum(c_jkm[j][0][m] for m in range(m_len)) for j in range(j_num)), "subjob 1 ends before subjob2 starts")

    p1.optimize()
    
    
    ''' stage 2 '''
    p1_2 = Model("p1_2")
    
    # set time limit
    p1_2.setParam('TimeLimit', time_limit2)
    j_num = int(len(job_id))
    m_len = len(machine_set)

    # tardy job num result from part 1
    T = p1.objVal

    # Variables
    u = p1_2.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "u")
    s_jkm = []
    c_jkm = []
    x_jkm = []
    for j in range(j_num):
        tmp_s = []
        tmp_c = []
        tmp_x = []
        for k in range(2):
            tmp_s.append([])
            tmp_c.append([])
            tmp_x.append([])
            for m in machine_set:
                tmp_s[k].append(p1_2.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "s_" + str(j + 1) + "," + str(k + 1) + "," + str(m)))
                tmp_c[k].append(p1_2.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "c_" + str(j + 1) + "," + str(k + 1) + "," + str(m)))
                tmp_x[k].append(p1_2.addVar(lb = 0, vtype = GRB.BINARY, name = "x_" + str(j + 1) + "," + str(k) + ","+ str(m)))  
        s_jkm.append(tmp_s)
        c_jkm.append(tmp_c)
        x_jkm.append(tmp_x)
#     print(len(x_jkm))
#     print(len(x_jkm[0]))
#     print(len(x_jkm[0][0]))

    y_jkjkm = []
    for j1 in range(j_num):
        y_jkjkm.append([])
        for k1 in range(2):
            y_jkjkm[j1].append([])
            for j2 in range(j1 + 1, j_num):
                y_jkjkm[j1][k1].append([])
                for k2 in range(2):
                    y_jkjkm[j1][k1][j2 - j1 - 1].append([])
                    for m in machine_set:
                        y_jkjkm[j1][k1][j2 - j1 - 1][k2].append(p1_2.addVar(lb = 0, vtype = GRB.BINARY, 
                                                       name = "y_" + str(j1 + 1) + "," + str(k1 + 1) + "," + str(j2 + 1) + "," + str(k2 + 1)+ "," + str(m)))

    c_j = []
    for j in range(j_num):
        c_j.append(p1_2.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "c_" + str(j + 1)))
    t_j = []
    for j in range(j_num):
        t_j.append(p1_2.addVar(lb = 0, vtype = GRB.BINARY, name = "t_" + str(j + 1)))
    
        
        
    # setting the objective function 
    p1_2.setObjective(u, GRB.MINIMIZE) 
    
    
    # add constraints and name them
    # makespan
    p1_2.addConstrs((u >= c_j[j] for j in range(j_num)))
    
    # tardy jobs num set to "lesser than or equal to" T
    p1_2.addConstr((quicksum(t_j[j] for j in range(j_num)) <= T))

    # stage 1 machine limit
    p1_2.addConstrs((x_jkm[j][0][m] <= m_dummy_stage1[j][m] for j in range(j_num) for m in range(m_len)))
    # stage 2 machine limit
    p1_2.addConstrs((x_jkm[j][1][m] <= m_dummy_stage2[j][m] for j in range(j_num) for m in range(m_len)))

    # tardy variable
    p1_2.addConstrs((t_j[j] * L >= c_j[j] - due_time[j] 
                    for j in range(j_num)), "tardy count")

    # job complete time
    p1_2.addConstrs((c_j[j] >= quicksum(c_jkm[j][1][m] for m in range(m_len))
                    for j in range(j_num)), "job completion time")

    # each job assigned once
    p1_2.addConstrs((quicksum(x_jkm[j][k][m] for m in range(m_len)) == 1 for j in range(j_num) for k in range(2)), "job assignment constraint")

    p1_2.addConstrs((s_jkm[j][k][m] + c_jkm[j][k][m] <= x_jkm[j][k][m] * L for j in range(j_num) for k in range(2) for m in range(m_len)))

    # process time
    p1_2.addConstrs((c_jkm[j][0][m] >= s_jkm[j][0][m] + pt_stage1[j] - (1 - x_jkm[j][0][m]) * L for j in range(j_num) for m in range(m_len)))
    p1_2.addConstrs((c_jkm[j][1][m] >= s_jkm[j][1][m] + pt_stage2[j] - (1 - x_jkm[j][1][m]) * L for j in range(j_num) for m in range(m_len)))

    for m in range(m_len):
        for j1 in range(j_num):
            for k1 in range(2):
                for j2 in range(j_num - j1 - 1):
                    for k2 in range(2):
                    # 注意ijm的indexing方法不同
                        p1_2.addConstr(s_jkm[j1][k1][m] >= c_jkm[j1 + j2 + 1][k2][m] - y_jkjkm[j1][k1][j2][k2][m] * L)
                        p1_2.addConstr(s_jkm[j1 + j2 + 1][k2][m] >= c_jkm[j1][k1][m] - (1 - y_jkjkm[j1][k1][j2][k2][m]) * L)

    # subjob 2 starts only after subjob 1
    p1_2.addConstrs((quicksum(s_jkm[j][1][m] for m in range(m_len)) >= quicksum(c_jkm[j][0][m] for m in range(m_len)) for j in range(j_num)), "subjob 1 ends before subjob2 starts")
    p1_2.optimize()
    
    
    machine = np.zeros(shape=(j_num, 2))
    completion_time = np.zeros(shape=(j_num, 2))
    for j in range(j_num):
        for k in range(2):
            for m in range(m_len):
                if c_jkm[j][k][m].x > 0:
                    completion_time[j][k] = c_jkm[j][k][m].x
                    machine[j][k] = m + 1
    tardy = []
    for j in range(j_num):
        if t_j[j].x != 0:
            tardy.append(j + 1)
    
    return p1.objVal, p1_2.objVal, machine, completion_time, tardy

## Set instances and test

In [14]:
# set instances to test
path_list = ["instance 1.csv", "instance 2.csv", "instance 3.csv", "instance 4.csv", "instance 5.csv"]
path_list = ["instance 2.csv", "instance 3.csv", "instance 4.csv", "instance 5.csv"]
# path_list = ["instance 187.csv"]

In [15]:
tardy_list = []
makespan_list = []
machine_list = []
completion_time_list = []
tardy_job = []

for path in path_list:
    df, machine_set, dummyM1_col, dummyM2_col = file_preprocess(path)
    tardy, makespan, machine_tmp, completion_tmp, tardy_job_tmp= IP(df, machine_set, dummyM1_col, dummyM2_col, 60, 180)
    print("machine:", machine_tmp)
    print("completion time:", completion_tmp)
    tardy_list.append(tardy)
    makespan_list.append(makespan)
    machine_list.append(machine_tmp)
    completion_time_list.append(completion_tmp)
    tardy_job.append(tardy_job_tmp)

for i in range(len(path_list)):
    print("----------------")
    print("file:", path_list[i])
    print("number of tardy jobs:", tardy_list[i])
    print("tardy jobs:", tardy_job[i])
    print("makespan:", makespan_list[i])
    print("----------------")

Set parameter TimeLimit to value 60
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 2585 rows, 1452 columns and 7678 nonzeros
Model fingerprint: 0xc77de323
Variable types: 231 continuous, 1221 integer (1221 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+03]
Presolve removed 542 rows and 260 columns
Presolve time: 0.01s
Presolved: 2043 rows, 1192 columns, 6206 nonzeros
Variable types: 184 continuous, 1008 integer (1008 binary)
Found heuristic solution: objective 7.0000000

Root relaxation: objective 0.000000e+00, 320 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    2    7.00000    0.000


Root relaxation: objective 6.400000e+00, 320 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    6.40000    0   33          -    6.40000      -     -    0s
     0     0    6.40000    0   41          -    6.40000      -     -    0s
     0     0    6.40000    0   37          -    6.40000      -     -    0s
H    0     0                      14.6000000    6.40000  56.2%     -    0s
H    0     0                      14.0000000    6.40000  54.3%     -    0s
H    0     0                      11.4000000    6.40000  43.9%     -    0s
     0     0    6.40000    0   15   11.40000    6.40000  43.9%     -    0s
     0     0    6.40000    0   15   11.40000    6.40000  43.9%     -    0s
     0     0    6.40000    0   11   11.40000    6.40000  43.9%     -    0s
     0     0    6.40000    0   12   11.40000    6.40000  43.9%     -    0s
     0   

*14168  5726              56      20.5100000   16.46000  19.7%  38.0    7s
 17866  6525   19.98000   44    9   20.51000   16.46000  19.7%  42.6   10s
H18695  6114                      20.0400000   16.46000  17.9%  43.7   11s
*20472  5261              57      19.9800000   16.46000  17.6%  47.2   13s
 24050  4134   19.35000   46   55   19.98000   16.46000  17.6%  54.9   15s

Cutting planes:
  Gomory: 3
  Cover: 130
  Implied bound: 31
  Clique: 10
  MIR: 16
  StrongCG: 2
  Flow cover: 113
  Zero half: 1
  Network: 4
  RLT: 30
  Relax-and-lift: 26

Explored 34255 nodes (2240813 simplex iterations) in 18.38 seconds (31.19 work units)
Thread count was 10 (of 10 available processors)

Solution count 10: 19.98 20.04 20.51 ... 21.52

Optimal solution found (tolerance 1.00e-04)
Best objective 1.998000000000e+01, best bound 1.998000000000e+01, gap 0.0000%
machine: [[1. 5.]
 [4. 7.]
 [1. 1.]
 [1. 7.]
 [2. 6.]
 [3. 3.]
 [2. 1.]
 [5. 5.]
 [7. 7.]
 [6. 2.]
 [6. 7.]
 [4. 5.]
 [3. 3.]
 [7. 3.]
 [1. 6.

 80292 26263   18.60000   85   16   19.80000   15.90000  19.7%  37.9   35s
 91136 26483 infeasible   93        19.80000   15.90000  19.7%  40.0   40s
 101951 26037   15.90000  101   21   19.80000   15.90000  19.7%  42.2   45s
 112041 24613   15.90000   87   28   19.80000   15.90000  19.7%  43.5   50s
 123720 24179   16.80000   96   23   19.80000   15.90000  19.7%  44.8   55s
 137744 23187 infeasible   92        19.80000   15.90000  19.7%  46.0   60s
 146899 25014 infeasible   98        19.80000   15.90000  19.7%  46.6   65s
 158954 28287 infeasible   98        19.80000   15.90000  19.7%  47.2   70s
 171694 31825   15.90000   92   25   19.80000   15.90000  19.7%  48.0   75s
 182509 35084 infeasible  103        19.80000   15.90000  19.7%  48.4   80s
 191558 36672   15.90000  105   21   19.80000   15.90000  19.7%  49.0   85s
 204256 38677 infeasible  104        19.80000   15.90000  19.7%  50.0   90s
 215743 41835 infeasible  108        19.80000   15.90000  19.7%  50.7   95s
 227176 44324 

In [11]:
print("machine_list", machine_list)
print("completion_time_list", completion_time_list)

machine_list [array([[2., 7.],
       [3., 3.],
       [7., 7.],
       [4., 7.],
       [5., 6.],
       [1., 3.],
       [8., 4.],
       [9., 9.],
       [1., 9.],
       [9., 3.],
       [6., 5.],
       [2., 7.],
       [8., 3.]])]
completion_time_list [array([[12.3, 14.2],
       [ 7.5,  9.6],
       [ 4.3,  4.3],
       [ 7.2, 14.2],
       [ 9.3, 10.8],
       [11.6, 13.2],
       [ 3.7, 14. ],
       [11.4, 14.2],
       [ 1.8, 11.4],
       [ 2.9,  9.4],
       [ 9.4, 13.3],
       [ 4.6,  6. ],
       [ 8.7, 11.5]])]


In [12]:
import pickle
with open("../../examples/results/IP_result_instance187.pkl", "wb") as fp:   #Pickling
        pickle.dump([machine_list[0], completion_time_list[0]], fp)
        
# with open("../../examples/results/IP_result_instance187.pkl", "wb") as fp:   #Pickling
#         pickle.dump([machine_list[1], completion_time_list[1]], fp)

In [27]:
import pickle
with open("../../examples/results/IP_result_instance2.pkl", "wb") as fp:   #Pickling
        pickle.dump([machine_list[0], completion_time_list[0]], fp)
        
with open("../../examples/results/IP_result_instance3.pkl", "wb") as fp:   #Pickling
        pickle.dump([machine_list[1], completion_time_list[1]], fp)
        
with open("../../examples/results/IP_result_instance4.pkl", "wb") as fp:   #Pickling
        pickle.dump([machine_list[2], completion_time_list[2]], fp)

In [ ]:
for i in range():
    with open("../../results/IP_result_instance" + str(i + 1) + ".pkl", "wb") as fp:   #Pickling
        pickle.dump([machine_list[i], completion_time_list[i]], fp)